# Домашнее задание 2 (5 баллов).

Все задания ниже имеют равный вес (5/16).

In [207]:
import io
import os
import re
import zipfile

import requests
import pandas as pd

#### Описание данных

В папке Dat (https://github.com/hse-ds/iad-intro-ds/blob/master/2023/homeworks/Data.zip) находится информация о студентах. Всего 10 групп студентов. Файлы делятся на две категории:
    * Students_info_i - информация о студентах из группы i
    * Students_marks_i - оценки студентов из группы i за экзамены

### Одно из важных достоинств pandas $-$ это удобные методы реляционного взаимодействия с данными, аналогичные, например, возможностям SQL для слияния и конкатенации таблиц: merge, join, concat. Наличие готовых методов позволяет не реализовывать самостоятельно поэлементную обработку данных и оперировать сразу целыми таблицами данных.

Подробно об этих методах посмотрите тут: https://www.kaggle.com/residentmario/renaming-and-combining#Combining

#### Задание 1. Соберите всю информацию о студентах в одну таблицу df. В получившейся таблице должна быть информация и оценки всех студентов из всех групп. Напечатайте несколько строк таблицы для демонстрации результата.¶

In [208]:
# Download archive locally
url = r"https://github.com/hse-ds/iad-intro-ds/raw/master/2023/homeworks/Data.zip"
r = requests.get(url)
assert r.ok
zipfile.ZipFile(io.BytesIO(r.content)).extractall(".")

In [209]:
def create_students_table() -> pd.DataFrame:
    info_table = pd.concat([pd.read_csv(f"./Data/{fname}") for fname in os.listdir("./Data") if "info" in fname])
    info_table.set_index("index", inplace=True)
    marks_table = pd.concat([pd.read_csv(f"./Data/{fname}") for fname in os.listdir("./Data") if "mark" in fname])
    marks_table.set_index("index", inplace=True)
    res =  pd.merge(info_table, marks_table, left_index=True, right_index=True)
    res.sort_index()
    return res

In [210]:
df = create_students_table()

In [211]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,group,math score,reading score,writing score
index,,,,,,,,,
0,female,group B,bachelor's degree,standard,none,group1,72,72,74
1,female,group C,some college,standard,completed,group1,69,90,88
2,female,group B,master's degree,standard,none,group1,90,95,93
3,male,group A,associate's degree,free/reduced,none,group1,47,57,44
4,male,group C,some college,standard,none,group1,76,78,75


#### Задание 2. Удалите столбец index у полученной таблицы. Напечатайте первые 10 строк таблицы.

In [212]:
# Now "index" column is a dataframe's index. Reset it so it becomes a common column and then drop it
df.reset_index(inplace=True)
df.drop("index", axis=1, inplace=True)


#### Задание 3. Выведите на экран размеры полученной таблицы

In [213]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,group,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,group1,72,72,74
1,female,group C,some college,standard,completed,group1,69,90,88
2,female,group B,master's degree,standard,none,group1,90,95,93
3,male,group A,associate's degree,free/reduced,none,group1,47,57,44
4,male,group C,some college,standard,none,group1,76,78,75


#### Задание 4. Выведите на экран статистические характеристики числовых столбцов таблицы (минимум, максимум, среднее значение, стандартное отклонение)

In [214]:
for col in df:
    series = df[col]
    if not pd.api.types.is_numeric_dtype(series):
        continue

    print("Column '{}':\n"
          "   min = {}\n"
          "   max = {}\n"
          "   mean = {}\n"
          "   std = {}\n".format(
              col, series.min(), series.max(), series.mean(), series.std()))

Column 'math score':
   min = 0
   max = 100
   mean = 66.089
   std = 15.163080096009468

Column 'reading score':
   min = 17
   max = 100
   mean = 69.169
   std = 14.60019193725222

Column 'writing score':
   min = 10
   max = 100
   mean = 68.054
   std = 15.195657010869642



#### Задание 5. Проверьте, есть ли в таблице пропущенные значения

In [215]:
print(f"Numer of missing values: {df.isnull().sum().sum()}")

Numer of missing values: 0


#### Задание 6. Выведите на экран средние баллы студентов по каждому предмету (math, reading, writing)

In [216]:
partial_subject_names = re.compile(r"(math*)|(reading*)|(writing*)")
subject_scores = [str(col) for col in df if partial_subject_names.match(str(col))]

In [217]:
for score_type in subject_scores:
    print(f"average {score_type}: {df[score_type].mean()}\n")

average math score: 66.089

average reading score: 69.169

average writing score: 68.054



**Задание 7. Как зависят оценки от того, проходил ли студент курс для подготовки к сдаче экзамена (test preparation course)? Выведите на экран для каждого предмета в отдельности средний балл студентов, проходивших курс для подготовки к экзамену и не проходивших курс.**

In [218]:
df[subject_scores + ["test preparation course"]].groupby("test preparation course").std()

,math score,reading score,writing score
test preparation course,,,
completed,14.444699,13.638384,13.375335
none,15.192376,14.463885,14.999661


**Задание 8. Выведите на экран все различные значения из столбца lunch.**

In [219]:
print(*df["lunch"].unique(), sep='\n')

standard
free/reduced


**Задание 9. Переименуйте колонку "parental level of education" в "education", а "test preparation course" в "test preparation" с помощью метода pandas rename**
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html

In [220]:
df.rename(columns={
    "parental level of education": "education",
    "test preparation course": "test preparation"
}, inplace=True)
df.head()

,gender,race/ethnicity,education,lunch,test preparation,group,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,group1,72,72,74
1,female,group C,some college,standard,completed,group1,69,90,88
2,female,group B,master's degree,standard,none,group1,90,95,93
3,male,group A,associate's degree,free/reduced,none,group1,47,57,44
4,male,group C,some college,standard,none,group1,76,78,75


**Зафиксируем минимальный балл для сдачи экзамена**

In [221]:
passmark = 50

**Задание 10. Ответьте на вопросы:**
    * Какая доля студентов сдала экзамен по математике (passmark >= 50)?
    * Какая доля студентов, проходивших курс подготовки к экзамену, сдала экзамен по математике?
    * Какая доля женщин, не проходивших курс подготовки к экзамену, не сдала экзамен по математике? 

In [222]:
print(f"Proportion of students who passed math exam:\n"\
      f"{len(df[df['math score'] >= passmark]) / len(df)}")

df_with_prep = df[df['test preparation'] == 'completed']

print(f"Proportion of students passed math exam amongst the students who attended preparation course:\n"\
      f"{len(df_with_prep[df_with_prep['math score'] >= passmark]) / len(df_with_prep)}")

df_women_without_prep = df[(df['gender'] == 'female') & (df['test preparation'] == 'none')]
print(f"Proportion of females who didnt pass math exam amongst the females who didnt attend preparation course:\n"\
      f"{len(df_women_without_prep[df_women_without_prep['math score'] < passmark]) / len(df_women_without_prep)}")

Proportion of students who passed math exam:
0.865
Proportion of students passed math exam amongst the students who attended preparation course:
0.9217877094972067
Proportion of females who didnt pass math exam amongst the females who didnt attend preparation course:
0.20958083832335328


**Задание 11. С помощью groupby выполните задания ниже. Также выведите время выполнения каждого из заданий.**
    * Для каждой этнической группы выведите средний балл за экзамен по чтению
    * Для каждого уровня образования выведите минимальный балл за экзамен по письму

In [223]:
%%time
df.groupby("race/ethnicity")["reading score"].mean()

CPU times: total: 0 ns
Wall time: 2 ms


race/ethnicity
group A    64.674157
group B    67.352632
group C    69.103448
group D    70.030534
group E    73.028571
Name: reading score, dtype: float64

In [224]:
%%time
df.groupby("education")["writing score"].min()

CPU times: total: 0 ns
Wall time: 999 µs


education
associate's degree    35
bachelor's degree     38
high school           15
master's degree       46
some college          19
some high school      10
Name: writing score, dtype: int64

**Задание 12. Выполните задание 11 с помощью циклов. Сравните время выполнения.**

Первая часть задания при помощи циклов:

In [225]:
def average_reading_in_groups():
    race_ethnicity_index = df.columns.tolist().index("race/ethnicity")
    reading_score_index = df.columns.tolist().index("reading score")
    groups_sums_and_sizes = {
        race_ethn: [0, 0] for race_ethn in df["race/ethnicity"].unique()
    }
    for i in range(len(df)):
        pair = groups_sums_and_sizes[df.iloc[i, race_ethnicity_index]]
        pair[0] += df.iloc[i, reading_score_index]
        pair[1] += 1

    return {race_ethn_info[0]: race_ethn_info[1][0] / race_ethn_info[1][1] for race_ethn_info in groups_sums_and_sizes.items()}

In [226]:
for race_ethn, mean in average_reading_in_groups().items():
    print(f"{race_ethn}: {mean}")

group B: 67.35263157894737
group C: 69.10344827586206
group A: 64.67415730337079
group D: 70.03053435114504
group E: 73.02857142857142


In [227]:
%%time
average_reading_in_groups()

CPU times: total: 62.5 ms
Wall time: 61 ms


{'group B': 67.35263157894737,
 'group C': 69.10344827586206,
 'group A': 64.67415730337079,
 'group D': 70.03053435114504,
 'group E': 73.02857142857142}

Вторая часть задания при помощи циклов:

In [228]:
def min_writing_for_edu():
    education_index = df.columns.tolist().index("education")
    writing_score_index = df.columns.tolist().index("writing score")
    groups_sums_and_sizes = { edu_type: float("+inf") for edu_type in df["education"].unique() }
    for i in range(len(df)):
        edu_str = df.iloc[i, education_index]
        groups_sums_and_sizes[edu_str] = min(groups_sums_and_sizes[edu_str], df.iloc[i, writing_score_index])
    return groups_sums_and_sizes


In [229]:
for edu_type, min_value in min_writing_for_edu().items():
    print(f"{edu_type}: {min_value}")

bachelor's degree: 38
some college: 19
master's degree: 46
associate's degree: 35
high school: 15
some high school: 10


In [230]:
%%time
min_writing_for_edu()

CPU times: total: 62.5 ms
Wall time: 63 ms


{"bachelor's degree": 38,
 'some college': 19,
 "master's degree": 46,
 "associate's degree": 35,
 'high school': 15,
 'some high school': 10}

**Задание 13. Выведите на экран средние баллы студентов по каждому предмету в зависимости от пола и уровня образования. То есть должно получиться количество групп, равных 2 * (число уровней образования), и для каждой такой группы выыведите средний балл по каждому из предметов.**

Это можно сделать с помощью сводных таблиц (pivot_table):

https://www.kaggle.com/kamilpolak/tutorial-how-to-use-pivot-table-in-pandas

In [231]:
pd.pivot_table(df, index=["gender", "education"], values=subject_scores, aggfunc="mean")

math score  reading score  writing score
gender education                                                   
female associate's degree   65.250000      74.120690      74.000000
       bachelor's degree    68.349206      77.285714      78.380952
       high school          59.351064      68.202128      66.691489
       master's degree      66.500000      76.805556      77.638889
       some college         65.406780      73.550847      74.050847
       some high school     59.296703      69.109890      68.285714
male   associate's degree   70.764151      67.433962      65.405660
       bachelor's degree    70.581818      68.090909      67.654545
       high school          64.705882      61.480392      58.539216
       master's degree      74.826087      73.130435      72.608696
       some college         69.009259      64.990741      63.148148
       some high school     67.840909      64.693182      61.375000

#### Задание 14. Сколько студентов успешно сдали экзамен по математике?

Создайте новый столбец в таблице df под названием Math_PassStatus и запишите в него F, если студент не сдал экзамен по математике (балл за экзамен < passmark), и P иначе.

Посчитайте количество студентов, сдавших и не сдавших экзамен по математике.

Сделайте аналогичные шаги для экзаменов по чтению и письму.

In [232]:
# Add new columns:
passed_status_cols = []
for subject_name in subject_scores:
    new_column_subject_name = subject_name[:subject_name.find(' ')]
    new_column_subject_name = new_column_subject_name[0].upper() + new_column_subject_name[1:]
    new_column_subject_name = f"{new_column_subject_name}_PassStatus"
    df[new_column_subject_name] = df[subject_name].apply(lambda exam_score: 'F' if exam_score < passmark else 'P')
    passed_status_cols.append(new_column_subject_name)

In [233]:
df.head(2)

,gender,race/ethnicity,education,lunch,test preparation,group,math score,reading score,writing score,Math_PassStatus,Reading_PassStatus,Writing_PassStatus
0,female,group B,bachelor's degree,standard,none,group1,72,72,74,P,P,P
1,female,group C,some college,standard,completed,group1,69,90,88,P,P,P


In [234]:
for subject_score, passed_status_col in zip(subject_scores, passed_status_cols):
    subject_vl_cnts = df[passed_status_col].value_counts()
    subject_name = subject_score[:subject_score.find(' ')]
    print(f"Number of students who passed {subject_name}: {subject_vl_cnts['P']}")
    print(f"Number of students who didnt pass {subject_name}: {subject_vl_cnts['F']}")

Number of students who passed math: 865
Number of students who didnt pass math: 135
Number of students who passed reading: 910
Number of students who didnt pass reading: 90
Number of students who passed writing: 886
Number of students who didnt pass writing: 114


#### Задание 15. Сколько студентов успешно сдали все экзамены?

Создайте столбец OverAll_PassStatus и запишите в него для каждого студента 'F', если студент не сдал хотя бы один из трех экзаменов, а иначе 'P'.

Посчитайте количество студентов, которые сдали все экзамены.

In [235]:
df["OverAll_PassStatus"] = df.apply(lambda row: 'P' if all(row[passed_status_col] == 'P' for passed_status_col in passed_status_cols) else 'F', axis=1)
df["OverAll_PassStatus"].value_counts()['P']

812

#### Задание 16. Переведем баллы в оценки

### Система перевода баллов в оценки
####    больше 90 = A
####      80-90 = B
####      70-80 = C
####      60-70 = D
####      50-60 = E
####    меньше 50 = F (Fail)

Создайте вспомогательную функцию, которая будет по среднему баллу за три экзамена выставлять оценку студенту по данным выше критериям.

Создайте столбец Grade и запишите в него оценку каждого студента.

Выведите количество студентов, получивших каждую из оценок.

**В случае, если средний балл попадает на границу между оценками (т.е. равен ровно 60, 70 или 80 баллов), вы можете интерпретировать условие на своё усмотрение (т.е. можете поставить за 60 баллов оценку D, а можете - E).**

In [246]:
def GetGrade(average_mark):
    if average_mark >= 90:
        return 'A'
    if average_mark >= 80:
        return 'B'
    if average_mark >= 70:
        return 'C'
    if average_mark >= 60:
        return 'D'
    return 'E' if average_mark >= 50 else 'F'

def average(*args) -> float:
    return sum(args) / len(args)

df["Grade"] = df.apply(lambda row: GetGrade(average(*(row[subject_scores] for subject_scores in subject_scores))), axis=1)

In [247]:
df.head(2)

,gender,race/ethnicity,education,lunch,test preparation,group,math score,reading score,writing score,Math_PassStatus,Reading_PassStatus,Writing_PassStatus,OverAll_PassStatus,Grade
0,female,group B,bachelor's degree,standard,none,group1,72,72,74,P,P,P,P,C
1,female,group C,some college,standard,completed,group1,69,90,88,P,P,P,P,B


In [248]:
df["Grade"].value_counts()

Grade
C    261
D    256
E    182
B    146
F    103
A     52
Name: count, dtype: int64